# Table of Contents
- [Processing Functions](#Process-functions)
- [Import Data](#import-data)
  - [Histograms](#Histograms)
  - [Counts files](#counts)
- [Processing](#processing)
- [Plotting](#plotting)
  - [HCT116-Rpb1 degron treatment with TRP/IAA, Biological Replicate 1](#LP-88-HCT116-Rpb1-degron-Biological-Replicate-1)
  - [HCT116-Rpb1 degron treatment with TRP/IAA, Biological Replicate 2](#LP-91-HCT116-Rpb1-degron-Biological-Replicate-2)
  - [HCT116-Rpb1 degron treatment with TRP/IAA, Biological Replicates 3 and 4](#LP-95-HCT116-Rpb1-degron-Biological-Replicates-3-and-4)
- [Spike-in Normalize](#spike-in-normalize)
  - [dual norm with confidence intervals](#dual-norm-with-confidence-intervals)

---

In [8]:
library(tidyverse)
theme_set(theme_classic())
library(DescTools)
library(RColorBrewer)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Processing HOMER Histograms

The functions: `process_histograms_dm6` `process_histograms_sac3` and `process_histograms_hg38` work with histogram files generated with HOMER annotatePeaks with -hist option. More details are available [on the HOMER tutorial website](http://homer.ucsd.edu/homer/ngs/quantification.html).

Making the histogram: 

`annotatePeaks.pl [region_to_center_on] [genome_version] -size [size_to_plot] -hist [bp_resolution] -d [tag directories] > histogram_regions_genome_datasource.txt`

Example for a TSS histogram in the hg38 genome: 

`annotatePeaks.pl tss hg38 -size 4000 -hist 25 -d sample1-tagdir sample2-tagdir input1-tagdir input2-tagdir > hist_tss_hg38_samples12.txt`

Input file: histogram_regions_genome_datasource.txt 

Output: dataframe in tidy (long) format, with additional columns describing the metadata originally in each sample name:
- Cell Type
- Treatment
- Timepoint
- Biological Replicate
- Antibody (usually I have multiple IP samples from each biological sample/replicate)
- Technical Replicate (separate library preps from the same biological sample/replicate)

In [1]:
process_histograms_dm6 <- function(x, .x) {
    colnames(x)[1] <- "Distance_from_tss"
    x <- x %>% 
      ### first two rename_with uses are specific to my sample names
      # get rid of suffix to samples, leaving only sample name.Coverage
    rename_with(~ gsub("\\.+\\.Coverage", ".Coverage", .x), contains("tagdir")) %>% 
    rename_with(~ gsub("_H3K9ac", "_1_H3K9ac", .x), contains("Hela")) %>%
    rename_with(~ gsub("_input", "_1_input", .x), contains("sync")) %>%
      # get rid of any prefix before cell type (ex: directory path)
    rename_with(~ gsub(".+\\_HCT", "HCT", .x), contains("HCT")) %>%
    # get rid of any suffix (ex: S2, S4)
    rename_with(~ gsub("_S[0-9]+", "", .x), contains("Hela|HCT")) %>%
    # add techreps to inputs
    rename_with(~ gsub("input_R1_001.", "input_1.", .x), contains("HCT")) %>%
     rename_with(~ gsub("_R1_001", "", .x), contains("Hela")) %>%
    rename_with(~ gsub("input.", "input_1.", .x), contains("HCT")) %>%
      ### below two uses of rename_with are general to HOMER output
    rename_with(~ gsub("\\.[[:digit:]]$", "_minus", .x), contains("Tags")) %>% 
    rename_with(~ gsub("\\.\\.\\.", "_", .x), contains("Tags"))
    
    # select coverage columns only
    xcov <- x %>% select(contains("Coverage"))
    xcov$Distance_from_tss <- x$Distance_from_tss
    
    xcovlong <- 
    xcov %>% pivot_longer(
      cols = -"Distance_from_tss", 
      names_to = "Sample", 
      values_to = "Coverage")
    xcovlong <- xcovlong %>%
  mutate(
    cell = str_match(Sample, '([^_]+)(?:_[^_]+){5}$')[,2],
    treatment = str_match(Sample, '([^_]+)(?:_[^_]+){4}$')[,2],
    timepoint = str_remove(str_match(Sample, '([^_]+)(?:_[^_]+){3}$')[,2], "TSA"),
    biorep = str_match(Sample, '([^_]+)(?:_[^_]+){2}$')[,2],
    antibody = str_match(Sample, '([^_]+)(?:_[^_]+){1}$')[,2],
    techrep = str_match(Sample, '([^_]+)(?:_[^_]+){0}$')[,2]
  )
  }

process_histograms_sac3 <- function(x, .x) {
    colnames(x)[1] <- "Distance_from_tss"
    x <- x %>% 
      ### first two rename_with uses are specific to my sample names
      # get rid of suffix to samples, leaving only sample name.Coverage
    rename_with(~ gsub(".concat.+Coverage", ".Coverage", .x), contains("tagdir")) %>% 
  #  rename_with(~ gsub("_H3K9ac", "_1_H3K9ac", .x), contains("Hela")) %>%
 #   rename_with(~ gsub("_input", "_1_input", .x), contains("sync")) %>%
      # get rid of any prefix before cell type (ex: directory path)
    rename_with(~ gsub(".+\\_HCT", "HCT", .x), contains("HCT")) %>%
    # get rid of any suffix (ex: S2, S4)
    rename_with(~ gsub("_S[0-9]+", "", .x), contains("Hela|HCT")) %>%
    # add techreps to inputs
    rename_with(~ gsub("input.", "input_1.", .x), contains("HCT")) %>%
    rename_with(~ gsub("input_R1_001.", "input_1.", .x), contains("HCT")) %>%
     rename_with(~ gsub("_R1_001", "", .x), contains("Hela")) %>%
      ### below two uses of rename_with are general to HOMER output
    rename_with(~ gsub("\\.[[:digit:]]$", "_minus", .x), contains("Tags")) %>% 
    rename_with(~ gsub("\\.\\.\\.", "_", .x), contains("Tags"))
    
    # select coverage columns only
    xcov <- x %>% select(contains("Coverage"))
    xcov$Distance_from_tss <- x$Distance_from_tss
    
    xcovlong <- 
    xcov %>% pivot_longer(
      cols = -"Distance_from_tss", 
      names_to = "Sample", 
      values_to = "Coverage")
    xcovlong <- xcovlong %>%
  mutate(
    cell = str_match(Sample, '([^_]+)(?:_[^_]+){5}$')[,2],
    treatment = str_match(Sample, '([^_]+)(?:_[^_]+){4}$')[,2],
    timepoint = str_remove(str_match(Sample, '([^_]+)(?:_[^_]+){3}$')[,2], "TSA"),
    biorep = str_match(Sample, '([^_]+)(?:_[^_]+){2}$')[,2],
    antibody = str_match(Sample, '([^_]+)(?:_[^_]+){1}$')[,2],
    techrep = str_match(Sample, '([^_]+)(?:_[^_]+){0}$')[,2]
  )
  }

process_histograms_hg38 <- function(x, .x) {
    colnames(x)[1] <- "Distance_from_tss"
    x <- x %>% 
      ### first two rename_with uses are specific to my sample names
      # get rid of suffix to samples, leaving only sample name.Coverage
    rename_with(~ gsub(".concat.hg38.tagdir.Coverage", ".Coverage", .x), contains("HCT")) %>% 
    rename_with(~ gsub(".concat.dedup.hg38.filtered.tagdir", "", .x), contains("tagdir")) %>% 
      # get rid of any prefix before cell type (ex: directory path)
  #  rename_with(~ gsub("_H3K9ac", "_1_H3K9ac", .x), contains("Hela")) %>%
    rename_with(~ gsub("_input", "_1_input", .x), contains("sync")) %>%
    rename_with(~ gsub(".+\\_HCT", "HCT", .x), contains("HCT")) %>% 
     # get rid of any suffix (ex: S2, S4)
    rename_with(~ gsub("_S[0-9]+", "", .x), contains("Hela|HCT")) %>%
    # add techreps to inputs
#    rename_with(~ gsub("input.", "input_1.", .x), contains("HCT")) %>%
    rename_with(~ gsub("input_R1_001.", "input_1.", .x), contains("HCT")) %>%
     rename_with(~ gsub("_R1_001", "", .x), contains("Hela")) %>%
      ### below two uses of rename_with are general to HOMER output
    rename_with(~ gsub("\\.[[:digit:]]$", "_minus", .x), contains("Tags")) %>% 
    rename_with(~ gsub("\\.\\.\\.", "_", .x), contains("Tags"))
    
    # select coverage columns only
    xcov <- x %>% select(contains("Coverage"))
    xcov$Distance_from_tss <- x$Distance_from_tss
    
    xcovlong <- 
    xcov %>% pivot_longer(
      cols = -"Distance_from_tss", 
      names_to = "Sample", 
      values_to = "Coverage")
    
    xcovlong <- xcovlong %>%
  mutate(
    cell = str_match(Sample, '([^_]+)(?:_[^_]+){5}$')[,2],
    treatment = str_match(Sample, '([^_]+)(?:_[^_]+){4}$')[,2],
    timepoint = str_remove(str_match(Sample, '([^_]+)(?:_[^_]+){3}$')[,2], "TSA"),
    biorep = str_match(Sample, '([^_]+)(?:_[^_]+){2}$')[,2],
    antibody = str_match(Sample, '([^_]+)(?:_[^_]+){1}$')[,2],
    techrep = str_match(Sample, '([^_]+)(?:_[^_]+){0}$')[,2]
  )
  }

## Processing HOMER counts files

The function `process_counts_annotpeaks()` takes HOMER annotatePeaks files generated without the -hist option. Instead of generating average Coverage across all regions specified, the resulting annotated counts file is a matrix, with rows for each region. The first 19 columns are metadata describing each region, the remaining columns are the Read Normalized Tag Counts for each sample at each region.

Relevant Metadata Columns: 
1. **PeakID**
2. **Chr**
3. **Start**
4. **End**
5. Strand - strand that nearest annotated gene/TSS is located on, not necessarily strand of your data! Remember ChIP-seq/ATAC-seq is unstranded by definition. 
6. Not Used - ignore
7. Focus.Ratio.Region.Size
8. **Annotation** - if annotating on TSS regions, all will be promoter-TSS
9. Detailed Annotation
10. **Distance to TSS** - if annotating on TSS regions, all will be 0.

# import data

In [11]:
# LP 88 - HCT116-Rpb1 degron biological replicate 1
hist_tss_hg38_LP88 <- read.delim("/home/lahodge/data/experiments/LP_88_combined_fastqs/hg38_data/hg38_tagdirs/hist_tss_hg38_LP88.txt")
hist_K27ac_hg38_LP88 <- read.delim("/home/lahodge/data/experiments/LP_88_combined_fastqs/hg38_data/hg38_tagdirs/hist_H3K27acpeaks_hg38_LP88.txt")
hist_over100kb_hg38_LP88 <- read.delim("/home/lahodge/data/experiments/LP_88_combined_fastqs/hg38_data/hg38_tagdirs/hist_over100kbgenes_hg38_LP88.txt")

# LP 91 - HCT116-Rpb1 degron biological replicate 2, technical replicate 1/2
hist_shortrna_hg38_LP91 <- read.delim("/home/lahodge/data/experiments/LP_91/hg38_data/hg38_tagdirs/hist_5to20kb_hg38_LP91.txt")
hist_tss_hg38_LP91 <- read.delim("/home/lahodge/data/experiments/LP_91/hg38_data/hg38_tagdirs/hist_tss_hg38_LP91.txt")
hist_rna_hg38_LP91 <- read.delim("/home/lahodge/data/experiments/LP_91/hg38_data/hg38_tagdirs/hist_rna_hg38_LP91.txt")

# LP 95 - HCT116-Rpb1 degron biological replicates 3 and 4

In [10]:
hist_tss_hg38_LP88_long <- process_histograms_dm6(hist_tss_hg38_LP88)
hist_over100kb_hg38_LP88_long <- process_histograms_hg38(hist_over100kb_hg38_LP88)

hist_shortrna_hg38_LP91_long <- process_histograms_hg38(hist_shortrna_hg38_LP91)
hist_tss_hg38_LP91_long <- process_histograms_hg38(hist_tss_hg38_LP91)

## LP 88 HCT116 Rpb1 degron Biological Replicate 1

### *H. sapiens* data

We quantify signal at regions biologically relevant for each antibody:
- H3K27ac: at TSSs or H3K27ac peaks
- H3K4me3: at TSSs or H3K4me3 peaks
- H3K4me1: at H3K4me1 peaks
- H3K36me3: at gene bodies or H3K36me3 peaks
- Rpb1: at TSSs or Rpb1 peaks

## Scatterplot comparing treatment changes

In [ ]:
some_function2_FC <- function(df, quantile, xnum, xdenom, ynum, ydenom, regions, colorbrewer) {
  # extract sample conditions from xvariable name
    cell_type <- str_match(xnum, '([^_]+)(?:_[^_]+){5}$')[,2]
    ### x axis variables
    treatment_xnum <- str_match(xnum, '([^_]+)(?:_[^_]+){4}$')[,2]
    timepoint_xnum <- str_match(xnum, '([^_]+)(?:_[^_]+){3}$')[,2]
    treatment_xdenom <- str_match(xdenom, '([^_]+)(?:_[^_]+){4}$')[,2]
    timepoint_xdenom <- str_match(xdenom, '([^_]+)(?:_[^_]+){3}$')[,2]
    biorep_xnum <- str_match(xnum, '([^_]+)(?:_[^_]+){2}$')[,2]
    biorep_xdenom <- str_match(xdenom, '([^_]+)(?:_[^_]+){2}$')[,2]
    antibody_xnum <- str_match(xnum, '([^_]+)(?:_[^_]+){1}$')[,2]
    antibody_xdenom <- str_match(xdenom, '([^_]+)(?:_[^_]+){1}$')[,2]
    xnumcondition <- paste(antibody_xnum, treatment_xnum, timepoint_xnum)
    xdenomcondition <- paste(antibody_xdenom, treatment_xdenom, timepoint_xdenom)
    ### y axis variables
    treatment_ynum <- str_match(ynum, '([^_]+)(?:_[^_]+){4}$')[,2]
    timepoint_ynum <- str_match(ynum, '([^_]+)(?:_[^_]+){3}$')[,2]
    treatment_ydenom <- str_match(ydenom, '([^_]+)(?:_[^_]+){4}$')[,2]
    timepoint_ydenom <- str_match(ydenom, '([^_]+)(?:_[^_]+){3}$')[,2]
    biorep_ynum <- str_match(ynum, '([^_]+)(?:_[^_]+){2}$')[,2]
    biorep_ydenom <- str_match(ydenom, '([^_]+)(?:_[^_]+){2}$')[,2]
    antibody_ynum <- str_match(ynum, '([^_]+)(?:_[^_]+){1}$')[,2]
    antibody_ydenom <- str_match(ydenom, '([^_]+)(?:_[^_]+){1}$')[,2]
    ynumcondition <- paste(antibody_ynum, treatment_ynum, timepoint_ynum)
    ydenomcondition <- paste(antibody_ydenom, treatment_ydenom, timepoint_ydenom)
  # make sure regions and colorbrewer varaiables are interpretable
  regions <- rlang::ensym(regions)
  colorbrewer <- rlang::ensym(colorbrewer)
  
  # need to convert quantile column to a factor (not 1-4 numeric) for plotting colors to work
#  df[quantile] <- as.factor(df[, quantile]) 
  num_colors <- length(unique(df[["quantile"]]))
  
  # make the plot
ggplot(data = df) + 
    aes(x = .data[[xnum]]/.data[[xdenom]], y = (.data[[ynum]])/(.data[[ydenom]]), 
        color = as.factor(quantile)) + 
    scale_x_log10() + scale_y_log10() + 
    geom_hline(yintercept = 1) + geom_vline(xintercept = 1) + 
    geom_point(alpha = 0.05, stroke = NA) + 
    labs(
      title = paste("comparing", xnumcondition, "and", ynumcondition, "changes"), 
      subtitle = paste("in", cell_type, "at", regions),
      x = paste(treatment_xnum, timepoint_xnum, "/", treatment_xdenom, timepoint_xdenom), 
      y = paste(treatment_ynum, timepoint_ynum, "/", treatment_ydenom, timepoint_ydenom)) +
  scale_color_manual(
      values = colorRampPalette(brewer.pal(9, paste0(colorbrewer)))(7)[4:7]) + 
  theme(legend.position = "none")
}

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6.5)

some_function2_FC(counts_K27_hg38_dual %>%
                    filter(HCT116_DMSO_0hr_1_H3K27ac_1.Tag > 
                             mean(HCT116_DMSO_0hr_1_H3K27ac_1.Tag)), quantile, 
                  xnum = "HCT116_TRP_4hr_1_H3K27ac_1.Tag", 
                  xdenom = "HCT116_DMSO_0hr_1_H3K27ac_1.Tag", 
                  ynum = "HCT116_IAA_4hr_1_H3K27ac_1.Tag", 
                  ydenom = "HCT116_DMSO_0hr_1_H3K27ac_1.Tag", 
                  regions = "shared H3K27ac peaks", 
                  colorbrewer = "RdYlBu") +
  aes(color = log(abs(Distance.to.TSS))) + 
  geom_point(alpha = 0.6, stroke = NA, size = 2) + 
scale_color_viridis() +
  theme(legend.position = c(0.82, 0.22), 
       axis.text = element_text(size=20), 
        axis.title=element_text(size=20), 
       legend.text = element_text(size=15),
       legend.title = element_text(size=18)) 

In [ ]:
counts_K27_hg38_dual$Annotation <- gsub("\\(.*", "", as.character(counts_K27_hg38_dual$Annotation))
counts_K27_hg38_dual$Annotation <- gsub(" ", "", as.character(counts_K27_hg38_dual$Annotation))

In [ ]:
counts_K27_hg38_dual_clean <- counts_K27_hg38_dual %>%
  mutate(
    Annotation_Clean = case_when(
      Annotation %in% c("intron", "exon", "5'UTR", "3'UTR", "TTS") ~ "genic_regions",
      Annotation %in% c("Intergenic", "non-coding") ~ "distal_nongenic_regions",
      Annotation == "promoter-TSS" ~ "promoter-TSS",
      TRUE ~ Annotation  # fallback: keep original if not matched
    )
  )

## LP 91 HCT116 Rpb1 degron Biological Replicate 2

In [ ]:
quantiling_peaks <- function(counts_peaks, nbin = 4, condition = NULL) {
  # Average across all tag columns
  counts_peaks <- counts_peaks %>%
    mutate(avg_tags = rowMeans(select(., ends_with("Tag")), na.rm = TRUE))
  
  # If condition provided
  if (!is.null(condition)) {
    # Support multiple patterns combined with "&"
    # Example: condition = c("DMSO", "Rpb1") or "DMSO&Rpb1"
    if (length(condition) > 1) {
      pattern <- paste0("(?=.*", condition, ")", collapse = "")
    } else if (grepl("&", condition)) {
      parts <- unlist(strsplit(condition, "&"))
      pattern <- paste0("(?=.*", parts, ")", collapse = "")
    } else {
      pattern <- condition
    }
    
     # Use perl = TRUE for lookaheads
    matched_cols <- select(counts_peaks, matches(pattern, perl = TRUE))
    
    counts_peaks <- counts_peaks %>%
      mutate(avg_cond = rowMeans(matched_cols, na.rm = TRUE))
  } else {
    counts_peaks <- counts_peaks %>%
      mutate(avg_cond = avg_tags)
  }
  
  counts_peaks <- counts_peaks %>%
    arrange(desc(avg_cond)) %>%
    mutate(quantile = ntile(avg_cond, nbin))
  
  return(counts_peaks)
}

# HCT116 violins from counts

In [12]:
counts_H3K27ac_hg38_fly_quant <- counts_H3K27ac_hg38_fly_quant %>%
  mutate(
    Annotation_simplified = case_when(
      Annotation == "promoter-TSS" ~ "promoter-TSS",
      Annotation == "Intergenic" ~ "Intergenic",
      TRUE ~ "gene-regions"
    )
  ) %>%
mutate(promoter_prox = if_else(abs(Distance.to.TSS) < 3000, "proximal", "distal"))

ERROR: Error: object 'counts_H3K27ac_hg38_fly_quant' not found


In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)
violin_HCT116_H3K27ac_4hrTRP_distal_gene_vs_nongene <- ggplot(counts_H3K27ac_hg38_fly_quant %>% filter(quantile == 4)) + 
aes(x = interaction(Annotation_simplified,promoter_prox), 
    y = (avg_4hr_TRP/avg_0hr_TRP)) + 
geom_hline(yintercept = 1) + 
geom_violin(linewidth = 1.1, draw_quantiles = c(0.25, 0.5, 0.75)) + 
# geom_jitter(alpha = 0.3, stroke = NA) + 
scale_y_log10() + 
coord_cartesian(ylim = c(0.2, 3)) + 
theme(
  axis.text = element_text(size = 14), 
  axis.title = element_text(size = 14), 
  legend.title = element_text(size = 14),
  legend.text = element_text(size = 14),
  title = element_text(size = 16),
  strip.text = element_text(size = 14)   # facet label text size
) + 
labs(title = "Changes at H3K27ac Peaks in top quartile of untreated Rpb1 signal",
     x = "Annotation", y = "H3K27ac Fold Change after 4hr TRP") 

In [ ]:
violin_HCT116_H3K27ac_4hrTRP_distal_gene_vs_nongene

In [ ]:
counts_H3K27ac_hg38_fly_quant <- counts_H3K27ac_hg38_fly_quant %>% 
mutate(avg_4hr_IAA = (HCT116_IAA_4hr_1_H3K27ac_1.Tag + HCT116_IAA_4hr_1_H3K27ac_2.Tag + HCT116_IAA_4hr_2_H3K27ac_1.Tag + HCT116_IAA_4hr_2_H3K27ac_2.Tag)/4)

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)
violin_HCT116_H3K27ac_4hrIAA_distal_gene_vs_nongene <- ggplot(counts_H3K27ac_hg38_fly_quant %>% filter(quantile == 4)) + 
aes(x = interaction(Annotation_simplified, promoter_prox), 
    y = (avg_4hr_IAA/avg_0hr_TRP)) + 
geom_hline(yintercept = 1) + 
geom_violin(linewidth = 1.1, draw_quantiles = c(0.25, 0.5, 0.75)) + 
# geom_jitter(alpha = 0.3, stroke = NA) + 
scale_y_log10() + 
coord_cartesian(ylim = c(0.2, 3)) + 
theme(
  axis.text = element_text(size = 14), 
  axis.title = element_text(size = 14), 
  legend.title = element_text(size = 14),
  legend.text = element_text(size = 14),
  title = element_text(size = 16),
  strip.text = element_text(size = 14)   # facet label text size
) + 
labs(title = "Changes at H3K27ac Peaks in top quartile of untreated Rpb1 signal",
     x = "Annotation", y = "H3K27ac Fold Change after 4hr IAA") 

In [ ]:
violin_HCT116_H3K27ac_4hrIAA_distal_gene_vs_nongene